# Various Metrics

### mnist data를 학습 후, scikit-learn 라이브러리를 이용해서 accuracy 외에도 precision, recall, f1 score 등의 다양한 metric들을 구해본다

In [13]:
##tenssorflow==2.3 환경에서 구현 
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

(mnist_train, mnist_train_label), (mnist_test, mnist_test_label) = mnist.load_data() #mnist data road
#normalization 적용
mnist_train=mnist_train/255
mnist_test=mnist_test/255

mnist_train=tf.cast(mnist_train, tf.float32)
mnist_train=np.expand_dims(mnist_train, -1)
mnist_test=tf.cast(mnist_test, tf.float32)
mnist_test=np.expand_dims(mnist_test, -1)

#one hot encoding
train_labels = to_categorical(mnist_train_label, 10)
test_labels = to_categorical(mnist_test_label, 10) 

class_number = 10 

# keras로 모델 구성 후, 학습
inputs = tf.keras.Input(shape=(28,28,1))
x = tf.keras.layers.Conv2D(kernel_size=(3,3), filters=16, activation='relu',padding="same")(inputs)
x = tf.keras.layers.MaxPooling2D((2,2),strides=(1, 1))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32, activation='relu',padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2,2),strides=(1, 1))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, activation='relu',padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2,2),strides=(2, 2))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, activation='relu',padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2,2),strides=(2, 2))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, activation='relu',padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2,2),strides=(2, 2))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(class_number, activation="softmax")(x)
model = tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(mnist_train, train_labels, epochs=100,validation_split=0.2,verbose=0)
print("train finished")

train finished


In [14]:
#최종 train accuracy 측정 (validation 데이터 포함)
metrics=model.evaluate(mnist_train, train_labels, verbose=2)
print("Train accuracy:", metrics[1])

1875/1875 - 3s - loss: 0.0127 - accuracy: 0.9969
Train accuracy: 0.9969333410263062


In [31]:
#학습한 모델로 mnist test data predict
pred=model.predict(mnist_test)

In [40]:
from sklearn.metrics import classification_report
#scikit-learn의 classification_report를 이용해서 다양한 metric들 도출
target_names=['0','1','2','3','4','5','6','7','8','9'] #class name
print(classification_report(np.argmax(test_labels,axis=-1), np.argmax(pred,axis=-1),target_names = target_names))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      0.99      0.99      1135
           2       1.00      0.99      1.00      1032
           3       1.00      1.00      1.00      1010
           4       1.00      0.98      0.99       982
           5       0.99      0.99      0.99       892
           6       0.99      0.99      0.99       958
           7       0.99      1.00      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

